# Mockup of Users Table

In [ ]:
# Set up the environment with AWS credentials
import boto3
import uuid
session = boto3.Session(profile_name='default', region_name='us-east-1')
ddb = session.resource('dynamodb')
dynamo = session.client('dynamodb')

# Define the table name
table_name = 'users'

# Create the table
table = ddb.create_table(
    TableName=table_name,
    KeySchema=[
        {
            'AttributeName': 'email',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'username',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'email',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'username',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

print(table.table_status)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
print(table.table_status)

In [ ]:
# Add a user
user1 = table.put_item(
    Item={
        'user_id': str(uuid.uuid4().hex),
        'profile_image': 's3:/stinkbait/static/img/users/bsmith.png',
        'email': 'bsmith@gmail.com',
        'username': 'bsmith',
        'first_name': 'Bob',
        'last_name': 'Smith',
        'address': '123 Main St.',
        'city': 'Anytown',
        'state': 'TX',
        'zip': '12345',
        'country': 'USA',
        'phone': '123-456-7890',
        'age': 35,
        'account_type': 'standard_user',
        'organizations': ['org1', 'org2'],
        'campaigns': ['camp1', 'camp2'],
        'targets': ['target1', 'target2'],
        'implants': ['implant1', 'implant2']
    }
)

print(user1)

In [ ]:
# Get a user
get_user1 = table.get_item(
    TableName = table_name,
    Key={
        'email': 'bsmith@gmail.com',
        'username': 'bsmith'
    }
)
print(get_user1['Item'])

In [ ]:
update_user1 = table.update_item(
    Key={
        'email': 'bsmith@gmail.com',
        'username': 'bsmith'
    },
    UpdateExpression='SET age = :val1',
    ExpressionAttributeValues={
        ':val1': 36
    }
)

print(update_user1)

In [ ]:
with table.batch_writer() as batch:
    user2 = batch.put_item(
        Item={
            'user_id': str(uuid.uuid4().hex),
            'profile_image': 's3:/stinkbait/static/img/users/wguthrie.png',
            'email': 'wguthrie@hotmail.com',
            'username': 'wguthrie',
            'first_name': 'Woody',
            'last_name': 'Guthrie',
            'address': '100 American Way',
            'city': 'Folk City',
            'state': 'OK',
            'zip': '32159',
            'country': 'USA',
            'phone': '+1 270 704 8035',
            'age': 137,
            'account_type': 'adminstrator',
            'organizations': ['verve org', 'rembrandt corp.', 'acme corp.'],
            'campaigns': ['holiday campaign', 'summer campaign', 'layoffs campaign', 'election campaign', 'recession campaign'],
            'targets': ['jalvarez@gmail.com', 'adamsuwei@hotmail.com', 'chriskringle@yahoo.com', 'bobprince@aol.com', 'gandolf@outlook.com'],
            'implants': ['8asd89safdn213das', 'as8fd9dasf13n89adf', '8a789qn13n989jk', 'sd8918sdf0n89adf', 'as89ds8f9fasd9f8df']
        }
    )
    user3 = batch.put_item(
        Item={
            'user_id': str(uuid.uuid4().hex),
            'profile_image': 's3:/stinkbait/static/img/users/sparks.png',
            'email': 'sparks@gmail.com',
            'username': 'sparks',
            'first_name': 'Sarah',
            'last_name': 'Parks',
            'address': '715 Maple St.',
            'city': 'Bridgeport',
            'state': 'VA',
            'zip': '13578',
            'country': 'USA',
            'phone': '+1 270 704 8035',
            'age': 21,
            'account_type': 'standard_user',
            'organizations': ['acme corp.', 'rembrandt corp.'],
            'campaigns': ['holiday campaign', 'summer campaign', 'layoffs campaign'],
            'targets': ['jjabrahms@waldorf.com', 'susanparker@filmore.org', 'r10a@google.com'],
            'implants': ['as89q4bn13n89adf', '8a789qn13n989jk', 'sd8918sdf0n89adf']
        }
    )

print(user2, user3)

# Query Users table

In [ ]:
query = table.query(
    KeyConditionExpression=boto3.dynamodb.conditions.Key('email').eq('sparks@gmail.com')
)
print(query['Items'])

# Empty table

In [ ]:
# Delete items from the table

items = dynamo.scan(
    TableName=table_name
)
print(items['Items'])

for item in items['Items']:
    print(f'Deleting: {item}[email][S], {item}[username][S]')
    table.delete_item(
        Key={
            'email': item['email']['S'],
            'username': item['username']['S']
        }
    )

# Delete the table

In [ ]:
delete_response = dynamo.delete_table(
    TableName=table_name
)
print(delete_response)